In [1]:
import json
import re
import time
import requests
import numpy as np
import pandas as pd
import os
import glob
from tqdm import tqdm
from datetime import datetime
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import warnings
warnings.filterwarnings('ignore')
from random import choice

In [2]:
current_datetime = datetime.now().strftime("%d-%m-%Y")
month_now = datetime.now().month
day_now = datetime.now().day

In [3]:
way = os.getcwd()

# Cookies and Headers для парсинга скидки и цены

In [4]:
cookies_discount = {
    'tmr_lvid': '286726132090325c59b36aa6b5e4aeaa',
    'tmr_lvidTS': '1692163620235',
    '_ym_uid': '1692163620496112993',
    '_ym_d': '1692163620',
    'city_id': '118',
    'city_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'city_full_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'region_id': '090a0c31-3b91-49a3-aa35-0c784898170b',
    'region_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'region_subdomain': 'spb',
    '_tt_enable_cookie': '1',
    '_ttp': 'tbKEEA3IkGrgdKpaT6-HRNf0K8h',
    'ccart': 'off',
    'uxs_uid': '898ab7e0-3bf5-11ee-8efe-f5b770db12a1',
    'loggedin': '%D0%AE%D1%80%D0%B8%D0%B9',
    'cseid': 'f4935df6bdac0029832e99cef94a5bc2',
    'sessionid': 'nr24mlml3ze25rk734gkkmgx3yv9ou7x',
    'authorization': 'true',
    'mindboxDeviceUUID': 'c254fa3f-cfcd-4146-9778-65be21c6a4df',
    'directCrm-session': '%7B%22deviceGuid%22%3A%22c254fa3f-cfcd-4146-9778-65be21c6a4df%22%7D',
    'auid': '651660ce-a06e-4b17-bd6f-69feaf135b08:1r5lu8:WYtTZugo2xVE6SqAc5ZSp_W-n1s5ztPHYxPs8YIwGQA',
    '_ga_G74SVHM1ZG': 'GS1.2.1700824600.19.0.1700824600.60.0.0',
    '_ga_GRDJMMMRHQ': 'GS1.2.1700824600.33.0.1700824601.59.0.0',
    '_ga_GRSMPDQ6SE': 'GS1.2.1700824598.88.1.1700825516.60.0.0',
    '_ga_M9ZHPEJXBM': 'GS1.2.1700824599.88.1.1700825517.60.0.0',
    'qr-go-to-the-app-banner': '1',
    'sncid': 'b36ee46b-981d-11e5-add2-0050568f2465:1rBa43:O1Wy8KuwGDddSCM7NMHFsjGhdPaqxcjhdblvWWbVams',
    'clt': 'new_old',
    '_ym_isad': '2',
    '_gid': 'GA1.2.2131018395.1702274408',
    'city_auto_popup_shown': '1',
    '_ym_visorc': 'b',
    '_ga': 'GA1.2.333284150.1692163620',
    '_ga_HJNSJ6NG5J': 'GS1.1.1702276831.120.1.1702280271.20.0.0',
    'digi_uc': 'W1sidiIsIjM0OTMxNCIsMTcwMjI4MTIxMjA0Nl0sWyJ2IiwiODQwMjIiLDE3MDIyODAyMzEzNjZdLFsidiIsIjQ2Nzk3IiwxNzAyMjgwMTI4Mzc3XSxbInYiLCI5OTM3NTQiLDE3MDIyNzgwMTczMjddLFsidiIsIjE0NjUyMSIsMTcwMjI3NzI5NDExM10sWyJ2IiwiOTI2ODAiLDE3MDIyNzY5NDQ1NDRdLFsidiIsIjg4MjEzIiwxNzAxODU5MjE3OTg0XSxbInYiLCI5MTAyMSIsMTcwMTg1OTA2ODA2MF0sWyJ2IiwiOTg3MDUwIiwxNzAxNzc4NDc0NzY5XSxbInYiLCIyNTYyMTciLDE3MDEzNDIzNTc4NTRdXQ==',
}

headers_discount = {
    'AB-TESTS': '{}',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
    # 'Cookie': 'tmr_lvid=286726132090325c59b36aa6b5e4aeaa; tmr_lvidTS=1692163620235; _ym_uid=1692163620496112993; _ym_d=1692163620; city_id=118; city_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; city_full_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_id=090a0c31-3b91-49a3-aa35-0c784898170b; region_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_subdomain=spb; _tt_enable_cookie=1; _ttp=tbKEEA3IkGrgdKpaT6-HRNf0K8h; ccart=off; uxs_uid=898ab7e0-3bf5-11ee-8efe-f5b770db12a1; loggedin=%D0%AE%D1%80%D0%B8%D0%B9; cseid=f4935df6bdac0029832e99cef94a5bc2; sessionid=nr24mlml3ze25rk734gkkmgx3yv9ou7x; authorization=true; mindboxDeviceUUID=c254fa3f-cfcd-4146-9778-65be21c6a4df; directCrm-session=%7B%22deviceGuid%22%3A%22c254fa3f-cfcd-4146-9778-65be21c6a4df%22%7D; auid=651660ce-a06e-4b17-bd6f-69feaf135b08:1r5lu8:WYtTZugo2xVE6SqAc5ZSp_W-n1s5ztPHYxPs8YIwGQA; _ga_G74SVHM1ZG=GS1.2.1700824600.19.0.1700824600.60.0.0; _ga_GRDJMMMRHQ=GS1.2.1700824600.33.0.1700824601.59.0.0; _ga_GRSMPDQ6SE=GS1.2.1700824598.88.1.1700825516.60.0.0; _ga_M9ZHPEJXBM=GS1.2.1700824599.88.1.1700825517.60.0.0; qr-go-to-the-app-banner=1; sncid=b36ee46b-981d-11e5-add2-0050568f2465:1rBa43:O1Wy8KuwGDddSCM7NMHFsjGhdPaqxcjhdblvWWbVams; clt=new_old; _ym_isad=2; _gid=GA1.2.2131018395.1702274408; city_auto_popup_shown=1; _ym_visorc=b; _ga=GA1.2.333284150.1692163620; _ga_HJNSJ6NG5J=GS1.1.1702276831.120.1.1702280271.20.0.0; digi_uc=W1sidiIsIjM0OTMxNCIsMTcwMjI4MTIxMjA0Nl0sWyJ2IiwiODQwMjIiLDE3MDIyODAyMzEzNjZdLFsidiIsIjQ2Nzk3IiwxNzAyMjgwMTI4Mzc3XSxbInYiLCI5OTM3NTQiLDE3MDIyNzgwMTczMjddLFsidiIsIjE0NjUyMSIsMTcwMjI3NzI5NDExM10sWyJ2IiwiOTI2ODAiLDE3MDIyNzY5NDQ1NDRdLFsidiIsIjg4MjEzIiwxNzAxODU5MjE3OTg0XSxbInYiLCI5MTAyMSIsMTcwMTg1OTA2ODA2MF0sWyJ2IiwiOTg3MDUwIiwxNzAxNzc4NDc0NzY5XSxbInYiLCIyNTYyMTciLDE3MDEzNDIzNTc4NTRdXQ==',
    'Origin': 'https://spb.sunlight.net',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'X-Requested-With': 'SunlightFrontendApp',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

# params = {
#     'article': '35517',
#     'city_id': '118',
# }

# response = requests.get(
#     'https://api.sunlight.net/v6/catalog/products/card/purchase/',
#     params=params,
#     cookies=cookies_discount,
#     headers=headers_discount,
# )

# Сбор скидок и цен

In [5]:
def get_personal_price(table):
    data_price = []
    for art in tqdm(range(len(table['article']))):
        # prx = choice(proxy_list)
        params = {
            'article': table['article'][art],
            'city_id': '118'
        }
        try:
            price_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/prices/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
        )
        except:
            time.sleep(6)
            price_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/prices/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        try:
            discount_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/purchase/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        except:
            time.sleep(6)
            discount_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/purchase/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        discount = []
        try:
            price_old = price_json.json()['content']['default_price']['base_sum']
        except:
            price_old = np.nan
        try:
            price = price_json.json()['content']['default_price'][('result_sum')]
        except:
            price = np.nan
        try:
            price_personal = price_json.json()['content']['default_price']['personal_block']['value']
        except:
            price_personal = np.nan

        try:
            for i in range(len(discount_json.json()['content']['nameplates'])):
                discount.append(discount_json.json()['content']['nameplates'][i]['text']['label'])
        except:
            discount.append(np.nan)
            
        data_price.append({
            'article': table['article'][art],
            'price_old': price_old,
            'price': price,
            'price_personal': price_personal,
            'discount': discount

        })
    return data_price

# Сбор цены, названия, группы, артикула

In [6]:
def get_sunlight(soup):
    try:
        price = soup.find(class_="supreme-product-card__price-discount-price").text
        price = re.sub(r'\D', '', str(price))
    except:
        price = np.nan
    try:
        name_product = soup.find('h1', class_="supreme-product-card__info-name-title").text
    except:
        name_product = np.nan
    try:
        group_product = soup.find_all('span', {'property': 'name'})[1].text
    except:
        group_product = np.nan
    try:
        article_product = soup.find_all('div', class_="supreme-product-card__info-name-article")[0].find('span').text
        article_product = re.sub('\D', '', str(article_product))
    except:
        article_product = np.nan
    return price, name_product, group_product, article_product

# Сбор всех параметров словарем

In [7]:
def get_params(soup):
    names = []
    values = []
    leight = len(soup.find_all('span', {'class': 'supreme-product-card-description__item-text'}))
    list_names = soup.find_all('div', {'class': 'supreme-product-card-description__item-title'})[-leight:]
    for i in range(leight):
        try:
            nam = list_names[i].text
            names.append(nam.replace('\n', '').strip())
        except:
            names.append(np.nan)
        try:
            values.append(soup.find_all('span', {'class': 'supreme-product-card-description__item-text'})[i].text)
        except:
            values.append(np.nan)
    sunlight_params = dict(zip(names, values))
    return sunlight_params

# Сбор ссылок

In [8]:
def get_links(url_sunlight):
    home = 'https://sunlight.net'
    links = []
    pages_max = 250
    pages = int(pages_max + 1)
    for i in tqdm(range(1, pages)):
        url = f'{url_sunlight}&page={i}'
        resp = requests.get(url, headers={'User-Agent': UserAgent().chrome}, verify=False, timeout=70)
        soup = BeautifulSoup(resp.text, 'html.parser')
        for link in soup.find_all('div', class_="cl-item js-cl-item"):
            links.append(home + link.find('a', href=True)['href'])
    links = list(map(lambda x: x.replace('#product-reviews', ''), links))
    return links

# Общий сбор данных

In [9]:
def sunlight_parser(links):
    data = []
    click = 0
    for link_pars in tqdm(links):
        # prx = choice(proxy_list)
        try:
            resp = requests.get(link_pars, headers={'User-Agent': UserAgent().chrome}, timeout=70, verify=False)
        except:
            time.sleep(6)
            resp = requests.get(link_pars, headers={'User-Agent': UserAgent().chrome}, timeout=70, verify=False)
        good = BeautifulSoup(resp.text, 'lxml')
        data.append({"date": current_datetime,
                    "competitor": "Sunlight",
                    "name": get_sunlight(good)[1],
                    "group": get_sunlight(good)[2],
                    "params": get_params(good),
                    "price": get_sunlight(good)[0],
                    "article": get_sunlight(good)[3],
                    'url': link_pars
                    })
        if click == 50:
            click = 0
            time.sleep(0.5)
        else:
            continue
    data = pd.DataFrame(data)
    return data

In [10]:
links_sunlight = get_links('https://spb.sunlight.net/catalog/?proba=2494')

  0%|          | 0/250 [00:00<?, ?it/s]

100%|██████████| 250/250 [02:53<00:00,  1.44it/s]


In [11]:
b = pd.DataFrame(links_sunlight)

In [12]:
b = b.drop_duplicates()

In [13]:
b = b[0].tolist()

In [14]:
slices = 100
chunks = np.array_split(b, slices)

In [15]:
n = 0
for i in range(n, slices):
    n += 1
    print(f'начало парсинга {n} части ссылок')
    df = sunlight_parser(chunks[i])
    df.to_csv(rf'{way}\parsed_data\sunlight_{day_now}_{month_now}_{n}.csv', index=False)
    time.sleep(5)

начало парсинга 1 части ссылок


100%|██████████| 137/137 [02:11<00:00,  1.04it/s]


начало парсинга 2 части ссылок


100%|██████████| 137/137 [02:23<00:00,  1.05s/it]


начало парсинга 3 части ссылок


...

100%|██████████| 136/136 [03:05<00:00,  1.36s/it]


начало парсинга 99 части ссылок


100%|██████████| 136/136 [02:48<00:00,  1.24s/it]


начало парсинга 100 части ссылок


100%|██████████| 136/136 [04:14<00:00,  1.87s/it]


In [16]:
os.chdir(rf'{way}\parsed_data')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
len(all_filenames)

100

In [17]:
data = pd.DataFrame()
for name  in all_filenames:
    df = pd.read_csv(name)
    data = pd.concat([df, data], ignore_index=True)

In [18]:
data

,date,competitor,name,group,params,price,article,url
0,22-07-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': '00496-50-030001-01*', 'Материал из...",49990.0,358923.0,https://sunlight.net/catalog/neck_decoration_3...
1,22-07-2024,Sunlight,\n Золотое шейное украшение с фианитами\n,Шейное украшение,"{'Модель': '00465-50-030006-00*', 'Материал из...",50020.0,358922.0,https://sunlight.net/catalog/neck_decoration_3...
2,22-07-2024,Sunlight,\n Золотое шейное украшение с ситаллами\n,Шейное украшение,"{'Модель': '00504-50-030008-11*', 'Материал из...",79990.0,358634.0,https://sunlight.net/catalog/neck_decoration_3...
3,22-07-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': '2130001к*', 'Материал изделия': 'Р...",374400.0,358479.0,https://sunlight.net/catalog/neck_decoration_3...
4,22-07-2024,Sunlight,\n Золотое шейное украшение с фианитами\n,Шейное украшение,"{'Модель': '03-3676*', 'Материал изделия': 'Ро...",20480.0,356642.0,https://sunlight.net/catalog/neck_decoration_3...
...,...,...,...,...,...,...,...,...
13667,22-07-2024,Sunlight,\n Золотое кольцо c бриллиантами\n,Кольца,"{'Модель': '2804-K5W-02', 'Материал изделия': ...",69990.0,58133.0,https://sunlight.net/catalog/ring_58133.html
13668,22-07-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '7552-K5M-02', 'Материал изделия': ...",44990.0,280238.0,https://sunlight.net/catalog/ring_280238.html
13669,22-07-2024,Sunlight,\n Золотые серьги с бриллиантами\n,Серьги,"{'Модель': '70629-C5W-01', 'Материал изделия':...",12990.0,151456.0,https://sunlight.net/catalog/earring_151456.html
13670,22-07-2024,Sunlight,\n Гайтан с золотым замком\n,Колье,"{'Модель': '73613.1*', 'Материал изделия': 'Те...",12990.0,104816.0,https://sunlight.net/catalog/necklace_104816.html


In [19]:
data.to_csv(rf'{way}\data_without_price.csv', index = False)

In [20]:
# data = pd.read_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\data_without_price.csv')

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13672 entries, 0 to 13671
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        13672 non-null  object 
 1   competitor  13672 non-null  object 
 2   name        13665 non-null  object 
 3   group       13665 non-null  object 
 4   params      13672 non-null  object 
 5   price       13665 non-null  float64
 6   article     13665 non-null  float64
 7   url         13672 non-null  object 
dtypes: float64(2), object(6)
memory usage: 854.6+ KB


In [22]:
data.duplicated().sum()

0

In [23]:
data = data.drop_duplicates()

In [24]:
data.duplicated().sum()

0

In [25]:
data['article'].isna().sum()

7

In [26]:
data = data.dropna()

In [27]:
data['article'] = data['article'].astype('int') 

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13665 entries, 0 to 13671
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        13665 non-null  object 
 1   competitor  13665 non-null  object 
 2   name        13665 non-null  object 
 3   group       13665 non-null  object 
 4   params      13665 non-null  object 
 5   price       13665 non-null  float64
 6   article     13665 non-null  int32  
 7   url         13665 non-null  object 
dtypes: float64(1), int32(1), object(6)
memory usage: 907.4+ KB


In [29]:
test = np.array_split(data, 20)

In [30]:
test[0]

,date,competitor,name,group,params,price,article,url
0,22-07-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': '00496-50-030001-01*', 'Материал из...",49990.0,358923,https://sunlight.net/catalog/neck_decoration_3...
1,22-07-2024,Sunlight,\n Золотое шейное украшение с фианитами\n,Шейное украшение,"{'Модель': '00465-50-030006-00*', 'Материал из...",50020.0,358922,https://sunlight.net/catalog/neck_decoration_3...
2,22-07-2024,Sunlight,\n Золотое шейное украшение с ситаллами\n,Шейное украшение,"{'Модель': '00504-50-030008-11*', 'Материал из...",79990.0,358634,https://sunlight.net/catalog/neck_decoration_3...
3,22-07-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': '2130001к*', 'Материал изделия': 'Р...",374400.0,358479,https://sunlight.net/catalog/neck_decoration_3...
4,22-07-2024,Sunlight,\n Золотое шейное украшение с фианитами\n,Шейное украшение,"{'Модель': '03-3676*', 'Материал изделия': 'Ро...",20480.0,356642,https://sunlight.net/catalog/neck_decoration_3...
...,...,...,...,...,...,...,...,...
679,22-07-2024,Sunlight,\n Золотые серьги с фианитами\n,Серьги,"{'Модель': '34-52-0642-00*', 'Материал изделия...",15990.0,167767,https://sunlight.net/catalog/earring_167767.html
680,22-07-2024,Sunlight,\n Золотые серьги с бриллиантами\n,Серьги,"{'Модель': 'С-14021', 'Материал изделия': 'Жел...",159990.0,100297,https://sunlight.net/catalog/earring_100297.html
681,22-07-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '20025-K5R-01', 'Материал изделия':...",59990.0,92924,https://sunlight.net/catalog/ring_92924.html
682,22-07-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '01-2443 Р', 'Материал изделия': 'Р...",79990.0,61538,https://sunlight.net/catalog/ring_61538.html


In [31]:
way = r'C:\Users\Trenkin.Sergey\Desktop\notebooks\sunlight\parsed_data'
n = 0
for i in range(n, 20):
    n += 1
    print(f'начало парсинга {n} части ссылок')
    df = get_personal_price(test[i].reset_index())
    df = pd.DataFrame(df)
    df.to_csv(rf'{way}\\' + f'sunlight_all_data_{day_now}_{month_now}_{n}.csv', index=False)
    time.sleep(5)

начало парсинга 1 части ссылок


100%|██████████| 684/684 [07:19<00:00,  1.56it/s]


начало парсинга 2 части ссылок


100%|██████████| 684/684 [07:11<00:00,  1.58it/s]


начало парсинга 3 части ссылок


...

100%|██████████| 683/683 [08:36<00:00,  1.32it/s]


начало парсинга 19 части ссылок


100%|██████████| 683/683 [08:33<00:00,  1.33it/s]


начало парсинга 20 части ссылок


100%|██████████| 683/683 [08:42<00:00,  1.31it/s]


In [32]:
os.chdir(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\sunlight\parsed_data')
extension = 'csv'
all_filenames = [i for i in glob.glob('sunlight_all_data*.{}'.format(extension))]
len(all_filenames)

20

In [33]:
data

,date,competitor,name,group,params,price,article,url
0,22-07-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': '00496-50-030001-01*', 'Материал из...",49990.0,358923,https://sunlight.net/catalog/neck_decoration_3...
1,22-07-2024,Sunlight,\n Золотое шейное украшение с фианитами\n,Шейное украшение,"{'Модель': '00465-50-030006-00*', 'Материал из...",50020.0,358922,https://sunlight.net/catalog/neck_decoration_3...
2,22-07-2024,Sunlight,\n Золотое шейное украшение с ситаллами\n,Шейное украшение,"{'Модель': '00504-50-030008-11*', 'Материал из...",79990.0,358634,https://sunlight.net/catalog/neck_decoration_3...
3,22-07-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': '2130001к*', 'Материал изделия': 'Р...",374400.0,358479,https://sunlight.net/catalog/neck_decoration_3...
4,22-07-2024,Sunlight,\n Золотое шейное украшение с фианитами\n,Шейное украшение,"{'Модель': '03-3676*', 'Материал изделия': 'Ро...",20480.0,356642,https://sunlight.net/catalog/neck_decoration_3...
...,...,...,...,...,...,...,...,...
13667,22-07-2024,Sunlight,\n Золотое кольцо c бриллиантами\n,Кольца,"{'Модель': '2804-K5W-02', 'Материал изделия': ...",69990.0,58133,https://sunlight.net/catalog/ring_58133.html
13668,22-07-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '7552-K5M-02', 'Материал изделия': ...",44990.0,280238,https://sunlight.net/catalog/ring_280238.html
13669,22-07-2024,Sunlight,\n Золотые серьги с бриллиантами\n,Серьги,"{'Модель': '70629-C5W-01', 'Материал изделия':...",12990.0,151456,https://sunlight.net/catalog/earring_151456.html
13670,22-07-2024,Sunlight,\n Гайтан с золотым замком\n,Колье,"{'Модель': '73613.1*', 'Материал изделия': 'Те...",12990.0,104816,https://sunlight.net/catalog/necklace_104816.html


In [34]:
data_discount = pd.DataFrame()
for name  in all_filenames:
    df = pd.read_csv(name)
    data_discount = pd.concat([df, data_discount], ignore_index=True)

In [35]:
data_discount

,article,price_old,price,price_personal,discount
0,988355,51 980 ₽,25 990 ₽,15 594 ₽,"['Ещё -5 000₽ здесь! Код: КУПИТЬ до 23:59', 'Е..."
1,331910,NaN,22 019 ₽,13 211 ₽,['Ещё -10% здесь! Код: КРАСНЫЙ до 22.07']
2,181069,39 980 ₽,19 990 ₽,11 994 ₽,"['Ещё -500₽ здесь! Код: КУПИТЬ до 23:59', 'Ещё..."
3,180077,25 980 ₽,12 990 ₽,7 794 ₽,"['Ещё -2 000₽ здесь! Код: КУПИТЬ до 23:59', 'Е..."
4,179902,33 460 ₽,16 730 ₽,10 038 ₽,"['Ещё -3 500₽ здесь! Код: КУПИТЬ до 23:59', 'Е..."
...,...,...,...,...,...
13660,167767,NaN,15 990 ₽,9 594 ₽,['Ещё -10% здесь! Код: КРАСНЫЙ до 22.07']
13661,100297,319 980 ₽,159 990 ₽,129 990 ₽,['Ещё -10% здесь! Код: ВЫГОДА до 23:59']
13662,92924,NaN,59 990 ₽,35 994 ₽,['Ещё -10% здесь! Код: КРАСНЫЙ до 22.07']
13663,61538,NaN,79 990 ₽,49 990 ₽,['Ещё -15% здесь! Код: КРАСНЫЙ до 22.07']


In [36]:
data = data.merge(data_discount, on='article', how='left')

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13665 entries, 0 to 13664
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            13665 non-null  object 
 1   competitor      13665 non-null  object 
 2   name            13665 non-null  object 
 3   group           13665 non-null  object 
 4   params          13665 non-null  object 
 5   price_x         13665 non-null  float64
 6   article         13665 non-null  int32  
 7   url             13665 non-null  object 
 8   price_old       10570 non-null  object 
 9   price_y         13665 non-null  object 
 10  price_personal  13023 non-null  object 
 11  discount        13665 non-null  object 
dtypes: float64(1), int32(1), object(10)
memory usage: 1.2+ MB


In [38]:
data['discount'].value_counts()

discount
['Ещё -10% здесь! Код: КРАСНЫЙ до 22.07']                                                                                      1035
['Ещё -2 000₽ здесь! Код: КУПИТЬ до 23:59', 'Ещё -10% здесь! Код: ВЫГОДА до 23:59']                                             961
['Ещё -3 500₽ здесь! Код: КУПИТЬ до 23:59', 'Ещё -10% здесь! Код: ВЫГОДА до 23:59']                                             796
['Ещё -5 000₽ здесь! Код: КУПИТЬ до 23:59', 'Ещё -10% здесь! Код: ВЫГОДА до 23:59']                                             684
['Ещё -1 000₽ здесь! Код: КУПИТЬ до 23:59', 'Ещё -10% здесь! Код: ВЫГОДА до 23:59']                                             664
                                                                                                                               ... 
['Ещё -7 500₽ здесь! Код: КУПИТЬ до 23:59', 'Ещё -25% здесь! Код: ВЫГОДА до 23:59', 'Цены снижены! Ещё -15%, код: СЕГОДНЯ']       1
['Ещё -10% здесь! Код: ВЫГОДА до 23:59', 'Цены снижены! Ещё -15%, к

In [39]:
data.head(20)

,date,competitor,name,group,params,price_x,article,url,price_old,price_y,price_personal,discount
0,22-07-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': '00496-50-030001-01*', 'Материал из...",49990.0,358923,https://sunlight.net/catalog/neck_decoration_3...,99 980 ₽,49 990 ₽,29 994 ₽,"['Ещё -5 000₽ здесь! Код: КУПИТЬ до 23:59', 'Е..."
1,22-07-2024,Sunlight,\n Золотое шейное украшение с фианитами\n,Шейное украшение,"{'Модель': '00465-50-030006-00*', 'Материал из...",50020.0,358922,https://sunlight.net/catalog/neck_decoration_3...,100 040 ₽,50 020 ₽,30 012 ₽,"['Ещё -7 500₽ здесь! Код: КУПИТЬ до 23:59', 'Е..."
2,22-07-2024,Sunlight,\n Золотое шейное украшение с ситаллами\n,Шейное украшение,"{'Модель': '00504-50-030008-11*', 'Материал из...",79990.0,358634,https://sunlight.net/catalog/neck_decoration_3...,159 980 ₽,79 990 ₽,49 990 ₽,['Ещё -25% здесь! Код: ВЫГОДА до 23:59']
3,22-07-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': '2130001к*', 'Материал изделия': 'Р...",374400.0,358479,https://sunlight.net/catalog/neck_decoration_3...,748 800 ₽,374 400 ₽,344 400 ₽,"['Ещё -40 000₽ здесь! Код: КУПИТЬ до 23:59', '..."
4,22-07-2024,Sunlight,\n Золотое шейное украшение с фианитами\n,Шейное украшение,"{'Модель': '03-3676*', 'Материал изделия': 'Ро...",20480.0,356642,https://sunlight.net/catalog/neck_decoration_3...,40 960 ₽,20 480 ₽,12 288 ₽,"['Ещё -2 000₽ здесь! Код: КУПИТЬ до 23:59', 'Е..."
5,22-07-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': '3С-0003-ж*', 'Материал изделия': '...",122410.0,356386,https://sunlight.net/catalog/bracelets_356386....,244 820 ₽,122 410 ₽,92 410 ₽,"['Ещё -15 000₽ здесь! Код: КУПИТЬ до 23:59', '..."
6,22-07-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '10510501/1*', 'Материал изделия': ...",17920.0,355528,https://sunlight.net/catalog/bracelets_355528....,35 840 ₽,17 920 ₽,10 752 ₽,"['Ещё -2 000₽ здесь! Код: КУПИТЬ до 23:59', 'Е..."
7,22-07-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': '01Б112846Р*', 'Материал изделия': ...",175340.0,354901,https://sunlight.net/catalog/bracelets_354901....,350 680 ₽,175 340 ₽,145 340 ₽,"['Ещё -40 000₽ здесь! Код: КУПИТЬ до 23:59', '..."
8,22-07-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': 'ЖШВБ 4-11-18*', 'Материал изделия'...",89690.0,996990,https://sunlight.net/catalog/bracelets_996990....,179 380 ₽,89 690 ₽,59 690 ₽,"['Ещё -7 500₽ здесь! Код: КУПИТЬ до 23:59', 'Е..."
9,22-07-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': 'ЖШБ 1-3*', 'Материал изделия': 'Ро...",67320.0,996989,https://sunlight.net/catalog/bracelets_996989....,134 640 ₽,67 320 ₽,40 392 ₽,"['Ещё -10 000₽ здесь! Код: КУПИТЬ до 23:59', '..."


In [40]:
data.to_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\workspace\\' + f'sunlight_all_{day_now}_{month_now}_{datetime.today().year}.csv', index=False)

In [41]:
previous = pd.read_csv(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\parsed_data\sunlight_raw.csv')

In [42]:
data = pd.concat([previous, data], ignore_index=True)

In [43]:
data

,date,competitor,name,group,params,price_x,article,url,price_old,price_y,price_personal,discount
0,21-05-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '71409-K5W-02', 'Материал изделия':...",99990.0,985936,https://sunlight.net/catalog/ring_985936.html,199 980 ₽,99 990 ₽,82 990 ₽,['Ещё -35% по коду: ЗОЛОТО до 23:59']
1,21-05-2024,Sunlight,\n Золотой браслет с бриллиантами\n,Браслеты,"{'Модель': '73353-B5R-01', 'Материал изделия':...",399990.0,985849,https://sunlight.net/catalog/bracelets_985849....,799 980 ₽,399 990 ₽,382 990 ₽,"['Второе бесплатно, код:1=2 до 27.05']"
2,21-05-2024,Sunlight,\n Золотая цепь\n,Цепи,"{'Модель': '01Ц032974Д*', 'Материал изделия': ...",133820.0,343470,https://sunlight.net/catalog/chains_343470.html,267 640 ₽,133 820 ₽,116 820 ₽,['Ещё -35% по коду: ЗОЛОТО до 23:59']
3,21-05-2024,Sunlight,\n Золотая цепь\n,Цепи,"{'Модель': '01Ц012974Д*', 'Материал изделия': ...",148990.0,343469,https://sunlight.net/catalog/chains_343469.html,297 980 ₽,148 990 ₽,131 990 ₽,['Ещё -35% по коду: ЗОЛОТО до 23:59']
4,21-05-2024,Sunlight,\n Золотая цепь\n,Цепи,"{'Модель': '45-0029-10*', 'Материал изделия': ...",106200.0,343447,https://sunlight.net/catalog/chains_343447.html,212 400 ₽,106 200 ₽,89 200 ₽,['Ещё -35% по коду: ЗОЛОТО до 23:59']
...,...,...,...,...,...,...,...,...,...,...,...,...
122108,22-07-2024,Sunlight,\n Золотое кольцо c бриллиантами\n,Кольца,"{'Модель': '2804-K5W-02', 'Материал изделия': ...",69990.0,58133,https://sunlight.net/catalog/ring_58133.html,139 980 ₽,69 990 ₽,41 994 ₽,"['Ещё -10 000₽ здесь! Код: КУПИТЬ до 23:59', '..."
122109,22-07-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '7552-K5M-02', 'Материал изделия': ...",44990.0,280238,https://sunlight.net/catalog/ring_280238.html,89 980 ₽,44 990 ₽,26 994 ₽,"['Ещё -7 500₽ здесь! Код: КУПИТЬ до 23:59', 'Е..."
122110,22-07-2024,Sunlight,\n Золотые серьги с бриллиантами\n,Серьги,"{'Модель': '70629-C5W-01', 'Материал изделия':...",12990.0,151456,https://sunlight.net/catalog/earring_151456.html,NaN,12 990 ₽,NaN,['ХИТ-цена']
122111,22-07-2024,Sunlight,\n Гайтан с золотым замком\n,Колье,"{'Модель': '73613.1*', 'Материал изделия': 'Те...",12990.0,104816,https://sunlight.net/catalog/necklace_104816.html,NaN,12 990 ₽,NaN,['ХИТ-цена']


In [44]:
data.to_csv(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\parsed_data\sunlight_raw.csv', index=False)

In [45]:
def delete_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f'Ошибка при удалении файла {file_path}. {e}')

In [48]:
delete_files_in_folder(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\sunlight\parsed_data')